In [ ]:
!pip install vnstock groq python-dotenv pandas nest-asyncio uvicorn fastapi

In [ ]:
import re
from datetime import datetime, timedelta
import pandas as pd
import json
import os
import time

from vnstock import Company, Quote

# FastAPI & Pydantic
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Async + notebook patch
import nest_asyncio
import asyncio
import uvicorn
nest_asyncio.apply()

# Groq
from groq import Groq
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# === Cấu hình Groq ===
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

groq_client = Groq(api_key=GROQ_API_KEY)
GROQ_MODEL = "llama-3.1-8b-instant"

In [6]:
def extract_tickers(text: str) -> list:
    candidates = re.findall(r'\b[A-Z]{3}\b', text.upper())
    
    common_words = {"THE", "AND", "FOR", "VND", "THEO", "CAO", "THAP", "NGAY", "TUAN", "QUA"}
    tickers = [t for t in candidates if t not in common_words]
    
    return tickers

In [7]:
def df_to_json_safe(df):
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype.kind in "M":
            df2[col] = df2[col].astype(str)
        elif df2[col].dtype.kind in "f":
            df2[col] = df2[col].replace({pd.NA: None, float('nan'): None})
        elif df2[col].dtype.kind in "i":
            df2[col] = df2[col].astype(int)
        else:
            df2[col] = df2[col].astype(str)
    return df2.to_dict(orient="records")

In [ ]:
def ai_format_answer(intent: str, detail_type: str, symbol: str, raw_data) -> str:
    if isinstance(raw_data, pd.DataFrame):
        limited = raw_data.tail(5) if len(raw_data) > 5 else raw_data
        data_str = json.dumps(df_to_json_safe(limited), ensure_ascii=False, indent=2)
    elif isinstance(raw_data, dict):
        data_str = json.dumps(raw_data, ensure_ascii=False, indent=2)
    else:
        data_str = str(raw_data)

    if detail_type == "total_volume_summary":
        prompt = f"""
                    Bạn là trợ lý chứng khoán. Trả lời **chính xác, ngắn gọn**, chỉ dựa trên dữ liệu sau.
                    KHÔNG thêm thông tin ngoài dữ liệu. KHÔNG nói về giá, cổ đông, tỷ lệ.
                    Dữ liệu: {data_str}
                    Mẫu: "Tổng khối lượng giao dịch của mã VIC từ 2025-11-07 đến 2025-11-14 là 3.899.846 cổ phiếu."
                    Trả lời:
                """
    else:
        prompt = f"""
                    Bạn là trợ lý chứng khoán chuyên nghiệp tại Việt Nam.
                    Nhiệm vụ: Dựa **chỉ** trên dữ liệu dưới đây, viết **câu trả lời tiếng Việt ngắn gọn, tự nhiên**.
                    - Không bịa thông tin.
                    - Nếu dữ liệu trống, hãy nói "Không tìm thấy dữ liệu."
                    - Loại dữ liệu: {detail_type} của {symbol}
                    Dữ liệu:
                    {data_str}
                    Trả lời:
                """

    for attempt in range(3):
        try:
            response = groq_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=GROQ_MODEL,
                temperature=0.4,
                max_tokens=500,
                top_p=0.9
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if attempt == 2:
                return f"Lỗi Groq: {str(e)}"
            time.sleep(1 * (2 ** attempt))
    return "Không thể tạo phản hồi."

In [9]:
def parse_question(question: str):
    tickers = extract_tickers(question)
    question_lower = question.lower()

    company_keywords = {
        "overview": ["thông tin", "hồ sơ", "giới thiệu", "doanh nghiệp"],
        "shareholders": ["cổ đông", "cổ đông lớn"],
        "officers": ["lãnh đạo", "ban lãnh đạo"],
        "subsidiaries": ["công ty con", "công ty thành viên"],
        "events": ["sự kiện"],
        "news": ["tin tức"],
        "reports": ["báo cáo phân tích"],
        "ratio_summary": ["tình hình tài chính", "tài chính"],
        "trading_stats": ["thống kê giao dịch"]
    }

    intent = "price"
    detail_type = None

    for key, kws in company_keywords.items():
        if any(k in question_lower for k in kws):
            intent = "company"
            detail_type = key
            break

    start, end = None, None
    if intent == "price":
        m = re.search(r"từ\s*(\d{4}-\d{2}-\d{2})\s*đến\s*(\d{4}-\d{2}-\d{2})", question)
        if m:
            start = datetime.fromisoformat(m.group(1))
            end = datetime.fromisoformat(m.group(2))
        else:
            days = 90
            m_days = re.search(r"(\d+)\s*(?:ngày|ngay|tuần|tháng|tuan|thang)", question_lower)
            if m_days:
                num = int(m_days.group(1))
                unit = m_days.group(0)
                if "tuần" in unit or "tuan" in unit:
                    days = num * 7
                elif "tháng" in unit or "thang" in unit:
                    days = num * 30
                else:
                    days = num
            end = datetime.today()
            start = end - timedelta(days=days)

    if intent == "company" and detail_type is None:
        detail_type = "overview"
    if intent == "price" and detail_type is None:
        detail_type = "history"

    return tickers, intent, detail_type, start, end

In [ ]:
def handle_price_comparison(tickers: list, question: str, start, end):
    try:
        data = {}
        for ticker in tickers:
            try:
                quote = Quote(symbol=ticker, source="VCI")
                hist = quote.history(start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"))
                if hist.empty:
                    data[ticker] = None
                    continue

                hist.columns = [col.lower() for col in hist.columns]
                if 'open' not in hist.columns:
                    data[ticker] = None
                    continue

                avg_open = hist['open'].mean()
                data[ticker] = int(round(avg_open))  

            except Exception:
                data[ticker] = None

        valid_data = {k: v for k, v in data.items() if v is not None}
        if not valid_data:
            return {"answer": "Không có dữ liệu giá cho các mã được yêu cầu."}

        min_ticker = min(valid_data, key=valid_data.get)

        formatted_prices = {
            ticker: f"{price:,}".replace(",", ".") + " VND"
            for ticker, price in valid_data.items()
        }

        prompt = f"""
                    Bạn là trợ lý chứng khoán chuyên nghiệp tại Việt Nam.
                    Hãy trả lời câu hỏi sau bằng tiếng Việt, **theo đúng định dạng**:
                    "{question}"
                    Dữ liệu:
                    - Các mã: {', '.join(tickers)}
                    - Giá mở cửa trung bình (10 ngày qua):
                    {chr(10).join([f"  - {ticker}: {formatted_prices[ticker]}" for ticker in tickers if ticker in formatted_prices])}
                    Yêu cầu:
                    1. Liệt kê **đầy đủ giá mở cửa trung bình của TẤT CẢ các mã** (theo định dạng trên).
                    2. Sau đó, kết luận: "→ Mã [XXX] có giá mở cửa trung bình thấp nhất."
                    3. Không thêm giải thích, không bịa thông tin.
                    4. Số tiền phải có đủ chữ số (ví dụ: 34.000 VND, không phải 34 VND).
                    Trả lời:
                """
        response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=GROQ_MODEL,
            temperature=0.2,  
            max_tokens=300
        )
        return {"answer": response.choices[0].message.content.strip()}

    except Exception as e:
        return {"answer": f"Lỗi so sánh: {str(e)}"}

In [ ]:
app = FastAPI(title="Vietnam Stock Agent + Groq", version="2.0")

class QuestionRequest(BaseModel):
    question: str

@app.post("/ask")
def ask_agent(request: QuestionRequest):
    question = request.question
    tickers, intent, detail_type, start, end = parse_question(question)

    if not tickers:
        raise HTTPException(status_code=400, detail="Không tìm thấy mã chứng khoán.")

    q_lower = question.lower()

    if (
        len(tickers) > 1
        and ("giá" in q_lower or "mở cửa" in q_lower or "đóng cửa" in q_lower)
        and ("thấp nhất" in q_lower or "cao nhất" in q_lower or "so sánh" in q_lower)
    ):
        return handle_price_comparison(tickers, question, start, end)

    ticker = tickers[0] if len(tickers) == 1 else tickers[-1]

    try:
        if intent == "company":
            company = Company(symbol=ticker, source="VCI")
            if detail_type == "overview":
                data = company.overview()
            elif detail_type == "shareholders":
                data = company.shareholders()
            elif detail_type == "officers":
                data = company.officers(filter_by='working')
            elif detail_type == "subsidiaries":
                data = company.subsidiaries()
            elif detail_type == "events":
                data = company.events()
            elif detail_type == "news":
                data = company.news()
            elif detail_type == "reports":
                data = company.reports()
            elif detail_type == "ratio_summary":
                data = company.ratio_summary()
            elif detail_type == "trading_stats":
                data = company.trading_stats()
            else:
                data = company.overview()

            answer_text = ai_format_answer("company", detail_type, ticker, data)
            return {"answer": answer_text}

        elif intent == "price":
            quote = Quote(symbol=ticker, source="VCI")
            if detail_type == "history":
                hist = quote.history(start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"))
                if hist.empty:
                    return {"answer": f"Không có dữ liệu giá cho {ticker} từ {start.strftime('%Y-%m-%d')} đến {end.strftime('%Y-%m-%d')}."}

                hist.columns = [col.lower() for col in hist.columns]

                required_cols = ['time', 'open', 'high', 'low', 'close', 'volume']
                if not all(col in hist.columns for col in required_cols):
                    return {"answer": f"Dữ liệu giá cho {ticker} bị thiếu cột cần thiết."}

                display_df = hist[required_cols].tail(10).copy()

                def format_price(x):
                    try:
                        return f"{int(x):,}".replace(",", ".") + " VND"
                    except:
                        return str(x)

                def format_volume(x):
                    try:
                        return f"{int(x):,}".replace(",", ".")
                    except:
                        return str(x)

                for col in ['open', 'high', 'low', 'close']:
                    display_df[col] = display_df[col].apply(format_price)
                display_df['volume'] = display_df['volume'].apply(format_volume)

                table_lines = []

                header = f"{'Ngày':<12} {'Mở cửa':<12} {'Cao nhất':<12} {'Thấp nhất':<12} {'Đóng cửa':<12} {'KL (cổ phiếu)':<15}"
                table_lines.append(header)
                table_lines.append("-" * len(header))

                for _, row in display_df.iterrows():
                    line = f"{row['time']:<12} {row['open']:<12} {row['high']:<12} {row['low']:<12} {row['close']:<12} {row['volume']:<15}"
                    table_lines.append(line)
                
                table_str = "\n".join(table_lines)

                prompt = f"""
                        Bạn là trợ lý chứng khoán chuyên nghiệp tại Việt Nam.
                        Dựa **chỉ** trên bảng dữ liệu sau, hãy:
                        1. **In lại nguyên bảng** (giữ nguyên định dạng căn lề).
                        2. Sau đó, viết **1 câu tóm tắt ngắn** về diễn biến giá (chỉ mô tả, không dự đoán).
                        3. **Không thêm thông tin ngoài bảng**, không bịa số liệu.

                        Bảng giá {ticker} (từ {start.strftime('%Y-%m-%d')} đến {end.strftime('%Y-%m-%d')}):

                        {table_str}

                        Trả lời:
                        """
                try:
                    response = groq_client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt}],
                        model=GROQ_MODEL,
                        temperature=0.2,
                        max_tokens=800
                    )
                    return {"answer": response.choices[0].message.content.strip()}
                except Exception as e:
                    # Fallback: trả bảng trực tiếp nếu Groq lỗi
                    fallback = f"Bảng giá {ticker}:\n\n{table_str}"
                    return {"answer": fallback}

            elif detail_type == "intraday":
                data = quote.intraday()
                return {"answer": ai_format_answer("price", "intraday", ticker, data)}

            elif detail_type == "depth":
                data = quote.price_depth()
                return {"answer": ai_format_answer("price", "depth", ticker, data)}

            else:
                hist = quote.history(start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"))
                return {"answer": ai_format_answer("price", "history", ticker, hist)}

    except Exception as e:
        if "Invalid symbol" in str(e):
            raise HTTPException(status_code=400, detail=f"Mã '{ticker}' không tồn tại.")
        raise HTTPException(status_code=500, detail=f"Lỗi hệ thống: {str(e)}")

def process_question(question: str) -> str:
    """Xử lý câu hỏi và trả về câu trả lời (dành cho test trong notebook)"""
    tickers, intent, detail_type, start, end = parse_question(question)
    if not tickers:
        return "Không tìm thấy mã chứng khoán."

    if len(tickers) > 1 and cần_so_sánh(...):
        result = handle_price_comparison(tickers, question, start, end)
        return result["answer"]

    ticker = tickers[0] if len(tickers) == 1 else tickers[-1]
    
    return answer_text  

In [ ]:
async def run_app():
    config = uvicorn.Config(app, host="127.0.0.1", port=8000)
    server = uvicorn.Server(config)
    await server.serve()
asyncio.run(run_app())